# In-Position Jitter Test
This script will handle the required post-processing of data for the in-position jitter test, specified in ASME B5.64. The script will input the csv file generated in Automation 1 for this test and return a completed In-Position Jitter Test Report.

There is also the option to have this script run an in-position jitter test directly.

In [1]:
#Updated by TB 01/08/2024
##Added a dropdown to be able to select cap probe or internal feedback
##Removed dropdown for test axis to be able to key in axis name to match customer MCD
##Made all relevant UI cells initialization cells

import pandas as pd
from IPython.display import Javascript, display, HTML
from InPositionJitterCollection import jitter
import ipywidgets as widgets
from tkinter import Tk, filedialog
from traitlets import traitlets
import automation1 as a1
import sys
import matplotlib.pyplot as plt
import numpy as np
from fpdf import FPDF
import matplotlib.image as image
import time
import copy
import scipy.signal as signal
import scipy.integrate as integrate

sys.path.append('../move-and-settle')

#Reach to get AerotechFormat module



sys.path.append('../')
import a1data
from AerotechFormat import AerotechFormat


#Use this class to have buttons store data
class LoadedButton(widgets.Button):
    #Button that can hold a value
    def __init__(self, value = None, *args, **kwargs):
        super(LoadedButton,self).__init__(*args, **kwargs)
        #Create value attribute
        self.add_traits(value = traitlets.Any(value))
        

        
#Front End Widgets
#print('System Information \n')
custom_style = "<style>.custom-font { font-family: 'Bold', sans-serif; font-size: 16px; color: black; font-weight: bold }</style>"

display(HTML(custom_style))

sysinfo = "System Information:"

display(HTML(f"<p class='custom-font'>{sysinfo}</p>"))

Type_w = widgets.Text(description = 'System Type:', value = "Linear")
ModelNumber_w = widgets.Text(description = "Model Number:", value = "ANT130XY", style= {'description_width': 'initial'}) #System model number/name
SerialNumber_w = widgets.Text(
    value = "XX", #System serial number
    description = "Serial Number:",
    style= {'description_width': 'initial'}
)
MaxTravel_w = widgets.Text(
    value = "60 mm", #System travel length
    description = "Max Travel Length:",
    style= {'description_width': 'initial'}
)
MaxVelocity_w = widgets.Text(
    value = "15 mm/s", #System max velocity
    description = "Maximum Velocity:",
    style= {'description_width': 'initial'}
)
ControllerName_w = widgets.Text(
    value = "Automation 1",
    description = "Controller Name:",
    style= {'description_width': 'initial'}
)

display(ModelNumber_w,SerialNumber_w,ControllerName_w)

#print('Measurement Information \n')
measinfo = "Measurement Information:"

display(HTML(f"<p class='custom-font'>{measinfo}</p>"))

FunctionalPoint_w = widgets.Text(
    value = "X = 0 mm, Y = 0 mm, Z = 35 mm", #[X,Y,Z], Location of the functional point for the test
    description = "Functional Point:",
    style= {'description_width': 'initial'}
)
SystemLocation_w = widgets.Text(
    description = "System Location:",
    style= {'description_width': 'initial'}
)
Load_w = widgets.Text(
    description = "Load:",
    value = "0 kg"
)
PreTestConditions_w = widgets.Text(
    description = "Pre-test Conditions:",
    style= {'description_width': 'initial'}
)
Temperature_w = widgets.IntText(
    description = "Temperature ($^\circ$C):",
    style= {'description_width': 'initial'},
    value = 20,
)
#DAS_w = widgets.Text(
    #description = "Data Aquisition System:",
    #style= {'description_width': 'initial'}
#)

Comments_w = widgets.Text(
    description = "Comments:"
)
display(FunctionalPoint_w, SystemLocation_w, Load_w,
       Temperature_w, Comments_w)


#print('Measurement Parameters\n')
measparam = "Measurement Parameters:"

display(HTML(f"<p class='custom-font'>{measparam}</p>"))

warning_style = "<style>.custom-font2 { font-family: 'Bold', sans-serif; font-size: 14px; color: red; font-weight: bold }</style>"

display(HTML(warning_style))

warning = "**Fields in this section are critical when running the Jitter test and when importing/post-processing data:"

display(HTML(f"<p class='custom-font2'>{warning}</p>"))

#Parameter imports are not quite working yet

##Option to upload Measurement parameters directly
#def selectParams(b):
#    root = Tk()
#    root.withdraw()                                        # Hide the main window.
#    root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
#    b.files = filedialog.askopenfilename(multiple=False)    # Selected file will be set button's file attribute.
#    b.value = b.files
#    print("Parameters Read")

#ParamPath = LoadedButton(
#    description = "Select Parameters",
#    style= {'description_width': 'initial'},
#)

#ParamPath.on_click(selectParams)

#display(ParamPath)
    


Units_w = widgets.Dropdown(options = [('nm','nm'), ('µm','um'),('mm', 'mm'),('m', 'm'),('deg','deg')], 
                           description = "Units (Must match controller):",
                           style= {'description_width': 'initial'},
                           value = 'mm')
Sensitivity = widgets.BoundedFloatText(
    description = 'Sensitivity (Units/V)',
    min = 0,
    value = 0.00254,
    style= {'description_width': 'initial'}
)
Axis = widgets.Text(
    description = 'Axis:',
    options = '',
    style= {'description_width': 'initial'}
)
ProbeAxis_w = widgets.Text(
    description = "Cap Probe Axis:",
    style= {'description_width': 'initial'},
    value = 'None'
)
Signal_w = widgets.Dropdown(
    description = "Feedback Signal:",
    style= {'description_width': 'initial'},
    options = (("Capacitance Probe", a1data.mode.ai0),("Internal Feedback",a1data.mode.pos_fbk))
)
Sensor = widgets.Text(
    description = "Sensor Name:",
    style= {'description_width': 'initial'},
    value = "Capacitance Probe"
)
SamplingRate_w= widgets.Dropdown(
    description = "Sample Rate:",
    options = [('1 kHz',1000), ('10 kHz', 10000) , ('20 kHz', 20000), ('100 kHz', 100000), ('200 kHz',200000)] #Available sample rates in Automation 1 
)
TestTime_w = widgets.FloatText(
    description = "Test Time (sec):",
    style= {'description_width': 'initial'},
    value = 10
)
Direction_w = widgets.Dropdown(
    description = "Direction to sensor:",
    style= {'description_width': 'initial'},
    options = (("Positive", a1data.mode.positive_direction),("Negative",a1data.mode.negative_direction))
)
TestPath = LoadedButton(
    description = "Run Jitter Test",
    layout = widgets.Layout(width = '20%')
)
CSVPath = LoadedButton(
    description = "Select File",
    style= {'description_width': 'initial'},
    layout = TestPath.layout
)
import_data_button = widgets.Button(
    description = 'Import Data',
    layout = widgets.Layout(width = '40%', height = '80%')
)

display(Units_w, Sensitivity, Axis, Sensor, Signal_w, ProbeAxis_w, SamplingRate_w, TestTime_w, Direction_w)


display(CSVPath, TestPath, import_data_button)


Comments = "" #Initialize Comments so it isn't a local variable

def selectFile(b):
    root = Tk()
    root.withdraw()                                        # Hide the main window.
    root.call('wm', 'attributes', '.', '-topmost', True)   # Raise the root to the top of all windows.
    b.files = filedialog.askopenfilename(multiple=False)    # Selected file will be set button's file attribute.
    b.value = pd.read_csv(r'{}'.format(b.files))
    

    CSVPath.description = 'File Selected'
    
def jittertest(b):
    print("Running jitter test...")
    controller = a1.Controller.connect()
    controller.start()
    
    for axis_index in range(0,1):
            
        # Create status item configuration object
        status_item_configuration = a1.StatusItemConfiguration()
            
        # Add this axis status word to object
        status_item_configuration.axis.add(a1.AxisStatusItem.AxisStatus, axis_index)

        # Get axis status word from controller
        result = controller.runtime.status.get_status_items(status_item_configuration)
        axis_status = int(result.axis.get(a1.AxisStatusItem.AxisStatus, axis_index).value)

        # Check NotVirtual bit of axis status word
        if (axis_status & 1<<13) == 0:
            controller = a1.Controller.connect_usb()
    global ipj
    ipj = jitter(Axis.value, SamplingRate_w.value, TestTime_w.value, Direction_w.value, Sensitivity.value, ProbeAxis_w.value, units = Units_w.value)
    ipj.test(controller)
    b.value = ipj.to_dataframe()
    display(Javascript("Jupyter.notebook.execute_cells([2])"))
    print('In-Position Jitter test completed')
    

                    
CSVPath.on_click(selectFile)
TestPath.on_click(jittertest)


def import_data(b):
    global ipj, Comments
    ipj = jitter(Axis.value, SamplingRate_w.value, TestTime_w.value, Direction_w.value, Sensitivity.value,ProbeAxis_w.value, units = Units_w.value)
    
#    #Update Parameters if a file exists
#    if ParamPath.value is not None:
#        ipj.import_params(ParamPath.value)
    
    if CSVPath.value is not None:
        ipj.populate(file = CSVPath.files)
        CSVPath.value = None
        CSVPath.description = "Select File"
    elif TestPath.value is not None:
        ipj.populate(dataframe = TestPath.value)
        TestPath.value = None
    else:
        raise ValueError('Please either run a jitter test or select a CSV file')
    print('Information successfully imported')

    display(Javascript("Jupyter.notebook.execute_cells([2,3])"))
    Comments = Comments_w.value
    
import_data_button.on_click(import_data)


Text(value='ANT130XY', description='Model Number:', style=TextStyle(description_width='initial'))

Text(value='XX', description='Serial Number:', style=TextStyle(description_width='initial'))

Text(value='Automation 1', description='Controller Name:', style=TextStyle(description_width='initial'))

Text(value='X = 0 mm, Y = 0 mm, Z = 35 mm', description='Functional Point:', style=TextStyle(description_width…

Text(value='', description='System Location:', style=TextStyle(description_width='initial'))

...

FloatText(value=10.0, description='Test Time (sec):', style=DescriptionStyle(description_width='initial'))

Dropdown(description='Direction to sensor:', options=(('Positive', <mode.positive_direction: 3>), ('Negative',…

LoadedButton(description='Select File', layout=Layout(width='20%'), style=ButtonStyle())

LoadedButton(description='Run Jitter Test', layout=Layout(width='20%'), style=ButtonStyle())

Button(description='Import Data', layout=Layout(height='80%', width='40%'), style=ButtonStyle())

In [ ]:
#print("Data Filters")
custom_style = "<style>.custom-font { font-family: 'Bold', sans-serif; font-size: 16px; color: black; font-weight: bold }</style>"

display(HTML(custom_style))

filters = "Data Filtering:"

display(HTML(f"<p class='custom-font'>{filters}</p>"))

reset = widgets.Button(
    description = "Reset Filters Applied"
)
display(reset)
print("Choose which data signal to filter, and then choose the filters to be applied")
mode = widgets.Dropdown(
    description = "Data Signal:",
    options = [("Position Feedback",a1data.mode.pos_fbk), ("Analog Input 0", a1data.mode.ai0)]
)
remove_offset = widgets.Button(
    description = "Remove DC Offset",
    layout = widgets.Layout(width = '40%', height = '80%')
)

end_norm = widgets.Button(
    description = "Endpoint Linear Normalization",
    layout = widgets.Layout(width = '40%', height = '80%')
)

square_norm = widgets.Button(
    description = "Least-Squares Linear Normalization",
    style= {'description_width': 'initial'},
    layout = widgets.Layout(width = '40%', height = '80%')
)
display(mode, remove_offset, end_norm, square_norm)
print("Butterworth Filter Parameters:")
omega_c = widgets.BoundedFloatText(
    description = "Cut-off Frequency (Hz):",
    style= {'description_width': 'initial'},
    value = 250,
    min = 0,
    max = 1000,
)
order = widgets.BoundedIntText(
    description = "Order:",
    value = 2,
    min = 1,
)
type_w = widgets.Dropdown(
    description = "Type:",
    options = [("High Pass",'high'),("Low Pass",'low')],
    value = 'low'
)
butter = widgets.Button(
    description = "Apply Filter"
)
display(omega_c,order,type_w,butter)

#Button Functions
def reset_func(b):
    global ipj, Comments
    ipj.pos_fbk = pos_fbk_copy
    ipj.ai0 = ai0_copy
    Comments = Comments_w.value
    print("Filters Reset")
    
def remove_offset_func(b):
    global ipj, Comments
    ipj.remove_offset(mode.value)
    Comments += "\nDC offset removed"
    print("DC offset removed")

def end_norm_func(b):
    global ipj, Comments
    ipj.endpoint_linear_norm(mode.value)
    Comments += "\nEndpoint Linear Normalization"
    
def square_norm_func(b):
    global ipj, Comments
    ipj.least_squares_linear_norm(mode.value)
    Comments += "\nLeast Squares Linear Normalization"
    
def butter_func(b):
    global ipj, Comments
    pos_fbk_copy = copy.deepcopy(ipj.pos_fbk)
    ai0_copy = copy.deepcopy(ipj.ai0)
    ipj.butter(mode.value, omega_c.value, order.value, type_w.value)
    Comments += '\n{} pass butterworth filter \nwith a {} Hz cut-off frequency \napplied'.format(type_w.value, omega_c.value)
    print("Butterworth Filter Applied")
    display(Javascript("Jupyter.notebook.execute_cells([3])"))
    
#Button Actions
reset.on_click(reset_func)
remove_offset.on_click(remove_offset_func)
end_norm.on_click(end_norm_func)
square_norm.on_click(square_norm_func)
butter.on_click(butter_func)


In [ ]:
#print("Plotting and Data Analysis")
custom_style = "<style>.custom-font { font-family: 'Bold', sans-serif; font-size: 16px; color: black; font-weight: bold }</style>"

display(HTML(custom_style))

analysis = "Plotting and Data Analysis:"

display(HTML(f"<p class='custom-font'>{analysis}</p>"))

low_bound = widgets.BoundedFloatText(
    description = "The low bound for the plot time window (sec):",
    min = 0,
    value = 0,
    style= {'description_width': 'initial'},
    layout = widgets.Layout(width = '35%')
)

high_bound = widgets.BoundedFloatText(
    description = "The high bound for the plot time window (sec):",
    min = low_bound.value,
    value = 1,
    style= {'description_width': 'initial'},
    layout = widgets.Layout(width = '35%')
)

CRMS_freq = widgets.BoundedFloatText(
    description = "Max Frequency for the CRMS plot (Hz):",
    min = 0,
    max = 1000,
    value = 200,
    style= {'description_width': 'initial'},

)

plot_data = widgets.Button(
    description = "Plot Data"
)

display(low_bound, high_bound, CRMS_freq, plot_data)

def plot_data_func(b):
    display(Javascript("Jupyter.notebook.execute_cells([4,5])"))
    
plot_data.on_click(plot_data_func)

    

In [ ]:
#Data Analysis
data_dict = ipj.data_analysis(mode.value, [low_bound.value, high_bound.value])
#Plotting the Jitter Data
plt.rcParams.update({'font.size': 40})
plt.figure(1, figsize = (20,10))
fig1 = plt.plot(data_dict['t_window'],data_dict['d_window'], '-r')

plt.xlabel('Time (seconds)')
plt.ylabel('Jitter ({})'.format(Units_w.value))
           
print('The sample standard deviation is {:.3e} {}'.format(data_dict['stdev'], Units_w.value))
print('The peak-to-peak in-position jitter is {:.3e} {}'.format(data_dict['peak'], Units_w.value))
           
plt.figure(3, figsize = (20,10))
           
plt.plot(data_dict['freq'],data_dict['CRMS'])
plt.xlabel('Frequency Hz')
plt.ylabel('Cumulative RMS ({})'.format(Units_w.value))

#Adjust frequency window for viewing the CRMS data
plt.xlim(0, CRMS_freq.value)



In [ ]:
#Output to pdf
custom_style = "<style>.custom-font { font-family: 'Bold', sans-serif; font-size: 16px; color: black; font-weight: bold }</style>"

display(HTML(custom_style))

pdf = "Generate PDF and CSV:"

display(HTML(f"<p class='custom-font'>{pdf}</p>"))

gen_PDF_button = widgets.Button(
    description = 'Generate PDF'
)

Units = Units_w.value
def gen_PDF(b):
    # This cell of the script will be used to generate a pdf in the AerotechFooter Format
    global fig
    
    plt.rcParams.update({'font.size': 6})
    fig, ax1, ax2, ax3, ax4 = AerotechFormat.makeTemplate()

    #Upper Jitter Plot
    ax1_up = plt.subplot2grid((14, 3),(2,0), rowspan = 3, colspan = 3)
    ax1_up.plot(data_dict['t_window'], data_dict['d_window'], '-r')
    plt.title('In-Position Jitter vs Time')
    plt.ylabel('Jitter ({})'.format(Units))
    plt.xlabel('Time (seconds)')

    #Lower Jitter Plot
    ax1_down = plt.subplot2grid((14,3),(6,0), rowspan = 3, colspan = 3)
    ax1_down.plot(data_dict['freq'],data_dict['CRMS'], '-b')
    plt.xlabel('Frequency Hz')
    plt.ylabel('Cumulative RMS ({})'.format(Units))
    plt.title('Cumulative RMS')
    plt.xlim(0,CRMS_freq.value)

    #Results Text Box
    ax2.text(0.02,.8, 'Standard Deviation: {:.3e} {}'.format(data_dict['stdev'], Units), color = 'black', size = 9)
    ax2.text(0.02,.725, 'Peak-to-Peak Value: {:.3e} {}'.format(data_dict['peak'], Units), color = 'black', size = 9)

    #Comments Text Box
    ax3.text(.02, .8, 'Serial Number: {}'.format(SerialNumber_w.value), color = 'black', size = 9)
    ax3.text(.02, .725, 'Model Number: {}'.format(ModelNumber_w.value), color = 'black', size = 9)
    ax3.text(.02, .65, 'Axis: {}'.format(Axis.value), color = 'black', size = 9)
    ax3.text(.02, .575, 'System Location: {}'.format(SystemLocation_w.value), color = 'black', size = 9)
    ax3.text(.02, .5, 'Signal: {}'.format(Sensor.value), color = 'black', size = 9)
    ax3.text(.02, .425, 'Comments: {}'.format(Comments), color = 'black', size = 6, verticalalignment = 'top')


    #Test Conditions Text Box
    degree_sign = u'\N{DEGREE SIGN}'
    ax4.text(.02, .8, 'Temperature: {}  {}C'.format(Temperature_w.value, degree_sign), color = 'black', size = 9)
    ax4.text(.02, .725, 'Sample Rate: {} Hz'.format(1/ipj.time_array[1]), color = 'black', size = 9)
    ax4.text(.02, .65, 'Sample Time: {} seconds'.format(np.max(ipj.time_array) + ipj.time_array[1]), color = 'black', size = 9)
    #ax4.text(.02, .725, 'Humidity: XX', color = 'black', size = 9)
    #ax4.text(.02, .65, 'Pressure: XX', color = 'black', size = 9)
    #ax4.text(.02, .575, 'Base Vibration: XX', color = 'black', size = 9)
    #ax4.text(.02, .350, 'Axis: {}'.format(Axis.value), color = 'black', size = 9)
    

gen_PDF_button.on_click(gen_PDF)

display(gen_PDF_button)

#Save PDF
output_file = widgets.Text(
    description = 'File Name:',
    value = 'jitter_report.pdf',
)

export_PDF_button = widgets.Button(
    description = 'Save Generated PDF'
)


def save_PDF(b):
    fig.get_figure().savefig(output_file.value)
    print('PDF saved')

export_PDF_button.on_click(save_PDF)
display(output_file, export_PDF_button)

#Save CSV Button and Textbox
csv_file = widgets.Text(
    description = 'CSV Name:',
    value = 'ipj.csv'
)

export_csv_button = widgets.Button(
    description = 'Save CSV'
)

def save_CSV(b):
    ipj.write_to_csv(csv_file.value)
    print('CSV saved')
    
export_csv_button.on_click(save_CSV)

display(csv_file, export_csv_button)